In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
STATE = 'California'
COUNTY = 'Orange'

DATA_DIR = 'nyt-data'


def calculate_daily_stats(df):
    df = df.set_index('date')
    df['daily_cases'] = df['cases'].diff().fillna(df['cases'])
    df['daily_deaths'] = df['deaths'].diff().fillna(df['deaths'])
    df['death_rate'] = df['deaths'] / df['cases']
    return df


country_df = pd.concat(
    [
        pd.read_csv(os.path.join(DATA_DIR, 'us.csv')),
        pd.read_csv(os.path.join(DATA_DIR, 'live', 'us.csv')),
    ],
    join='inner',
)
country_df = calculate_daily_stats(country_df)
state_df = pd.concat(
    [
        pd.read_csv(os.path.join(DATA_DIR, 'us-states.csv')),
        pd.read_csv(os.path.join(DATA_DIR, 'live', 'us-states.csv')),
    ],
    join='inner',
)
state_df = calculate_daily_stats(state_df[state_df['state'] == STATE])
county_df = pd.concat(
    [
        pd.read_csv(os.path.join(DATA_DIR, 'us-counties.csv')),
        pd.read_csv(os.path.join(DATA_DIR, 'live', 'us-counties.csv')),
    ],
    join='inner',
)
county_df = calculate_daily_stats(county_df[(county_df['state'] == STATE) & (county_df['county'] == COUNTY)])

print(country_df.tail())
print(state_df.tail())
print(county_df.tail())

In [ ]:
SIZE = (20, 12)

fig, axs = plt.subplots(3, 3, figsize=SIZE)


def draw_graphs(ax_row, df, region_label):
    ax_row[0].set(xticks=[], title=f'Daily Cases {region_label}')
    ax_row[0].bar(df.index, df['daily_cases'])
    ax_row[0].plot(df.index, df['daily_cases'].rolling(7, 1).mean(), color='red')
    
    ax_row[1].set(xticks=[], title=f'Daily Deaths {region_label}')
    ax_row[1].bar(df.index, df['daily_deaths'])
    ax_row[1].plot(df.index, df['daily_deaths'].rolling(7, 1).mean(), color='red')
    
    ax_row[2].set(xticks=[], title=f'Death Rate {region_label}')
    ax_row[2].plot(df.index, df['death_rate'])


draw_graphs(axs[0], country_df, 'U.S.')
draw_graphs(axs[1], state_df, f'State: {STATE}')
draw_graphs(axs[2], county_df, f'County: {COUNTY}')

plt.show()